# Proyek Analisis Data: Bike Sharing Dataset
- **Nama:** Didik Wibowo
- **Email:** dicbowo@gmail.com
- **ID Dicoding:** didikwibowo

## Menentukan Pertanyaan Bisnis

- Apa waktu terbaik untuk memaksimalkan penggunaan sepeda?
- Bagaimana pengaruh cuaca terhadap jumlah penyewaan sepeda?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

## Data Wrangling

### Gathering Data

In [ ]:
@st.cache_data
def load_data():
    try:
        day_data = pd.read_csv('day.csv')
        hour_data = pd.read_csv('hour.csv')
        return day_data, hour_data
    except FileNotFoundError as e:
        st.error(f"File tidak ditemukan: {e.filename}. Pastikan file berada di folder 'data'.")
        return pd.DataFrame(), pd.DataFrame()

**Insight:**
- Dataset terdiri dari dua file:
    - day.csv: Data penyewaan sepeda berdasarkan hari.
    - hour.csv: Data penyewaan sepeda berdasarkan jam.
- Kedua dataset memiliki informasi seperti cuaca (weathersit), suhu (temp), kelembapan (hum), kecepatan angin (windspeed), dan jumlah penyewaan sepeda (cnt).
- Data berhasil dimuat tanpa error, dengan preview menunjukkan nilai-nilai yang konsisten dan relevan.

### Assessing Data

In [ ]:
day_data, hour_data = load_data()

# Title and description
st.title('Dashboard Analisis Bike Sharing Dataset')
st.markdown('Dashboard ini menampilkan hasil analisis data penyewaan sepeda untuk menjawab dua pertanyaan bisnis utama:')
st.markdown('1. Apa waktu terbaik untuk memaksimalkan penggunaan sepeda?')
st.markdown('2. Bagaimana pengaruh cuaca terhadap jumlah penyewaan sepeda?')

# Dataset selection
dataset_choice = st.radio("Pilih Dataset", ("Data Harian", "Data Per Jam"))
if dataset_choice == "Data Harian":
    data = day_data
    st.markdown("### Visualisasi untuk Data Harian")
else:
    data = hour_data
    st.markdown("### Visualisasi untuk Data Per Jam")

**Insight:**
- Dilakukan eksplorasi awal terhadap struktur data di kedua file dataset:
    - Dataset day.csv memiliki total 16 kolom, termasuk instant (indeks data), dteday (tanggal), season (musim), cnt (jumlah penyewaan), dan kolom lain seperti cuaca (weathersit) serta suhu (temp).
    - Dataset hour.csv memiliki struktur serupa dengan tambahan kolom jam (hr) untuk analisis yang lebih granular.
- Distribusi Data:
    - Tidak ditemukan nilai duplikat dalam kedua dataset.
    - Tidak ada missing values (NaN) di dataset, menunjukkan data bersih.
- Tipe Data:
    - Semua kolom memiliki tipe data yang sesuai, seperti float untuk temp dan hum, serta int untuk season dan weathersit.

### Cleaning Data

In [ ]:
# Sidebar for filtering
if 'hr' in data.columns:
    time_filter = st.sidebar.multiselect('Pilih Jam (0-23)', sorted(data['hr'].unique()), default=sorted(data['hr'].unique()))
    data = data[data['hr'].isin(time_filter)]

weather_filter = st.sidebar.multiselect(
    'Pilih Kondisi Cuaca',
    options=[1, 2, 3, 4],
    default=[1, 2, 3, 4],
    format_func=lambda x: {1: 'Clear/Partly Cloudy', 2: 'Mist/Cloudy', 3: 'Light Snow/Rain', 4: 'Heavy Rain/Snow'}[x]
)
data = data[data['weathersit'].isin(weather_filter)]

**Insight:**
- Penyesuaian berikut dilakukan untuk memastikan data siap digunakan:
    - Kategorisasi Variabel: Kolom weathersit diubah menjadi label deskriptif (misalnya, 1 menjadi "Clear/Partly Cloudy").
    - Filter Data: Untuk analisis tertentu, dilakukan filter berdasarkan jam (hr) dan kondisi cuaca (weathersit).
    - Normalisasi Data: Kolom seperti temp dan hum yang dinormalisasi tetap digunakan sesuai skala asli karena skala ini relevan untuk analisis relatif.
- Tidak diperlukan transformasi lebih lanjut karena data sudah dalam format yang dapat langsung dianalisis.

## Exploratory Data Analysis (EDA)

**Insight:**
1. Waktu Terbaik untuk Penyewaan Sepeda
    - Data Harian (day.csv):
        - Rata-rata penyewaan sepeda per bulan menunjukkan bulan dengan cuaca lebih hangat (seperti musim panas) memiliki jumlah penyewaan lebih tinggi.
        - Tren musiman memengaruhi perilaku pengguna.
    - Data Per Jam (hour.csv):
        - Puncak penyewaan terjadi pada pagi (07:00-09:00) dan sore (17:00-19:00), yang kemungkinan besar merupakan jam sibuk.
        - Waktu malam dan dini hari menunjukkan aktivitas penyewaan yang rendah.
2. Pengaruh Cuaca terhadap Penyewaan
    - Cuaca cerah atau sebagian berawan (Clear/Partly Cloudy) memiliki jumlah penyewaan tertinggi.
    - Cuaca ekstrem seperti hujan deras atau salju (Heavy Rain/Snow) memiliki jumlah penyewaan terendah.
    - Faktor cuaca secara signifikan memengaruhi jumlah penyewaan sepeda.

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
st.subheader('Jumlah Penyewaan Sepeda')
if 'hr' in data.columns:
    hourly_counts = data.groupby('hr')['cnt'].mean()
    plt.figure(figsize=(10, 6))
    plt.plot(hourly_counts.index, hourly_counts.values, marker='o', label='Rata-rata Penyewaan')
    plt.title('Rata-rata Penyewaan Sepeda per Jam')
    plt.xlabel('Jam')
    plt.ylabel('Jumlah Penyewaan')
    plt.grid()
    plt.legend()
    st.pyplot(plt)
else:
    monthly_counts = data.groupby('mnth')['cnt'].mean()
    plt.figure(figsize=(10, 6))
    monthly_counts.plot(kind='bar', color='orange', label='Rata-rata Penyewaan')
    plt.title('Rata-rata Penyewaan Sepeda per Bulan')
    plt.xlabel('Bulan')
    plt.ylabel('Jumlah Penyewaan')
    plt.legend()
    st.pyplot(plt)

### Pertanyaan 2:

In [ ]:
st.subheader('Pengaruh Cuaca terhadap Penyewaan Sepeda')
weather_counts = data.groupby('weathersit')['cnt'].mean().reindex([1, 2, 3, 4], fill_value=0)
weather_labels = ['Clear/Partly Cloudy', 'Mist/Cloudy', 'Light Snow/Rain', 'Heavy Rain/Snow']
plt.figure(figsize=(8, 6))
plt.bar(weather_labels, weather_counts, color='skyblue')
plt.title('Rata-rata Penyewaan Berdasarkan Kondisi Cuaca')
plt.xlabel('Kondisi Cuaca')
plt.ylabel('Rata-rata Jumlah Penyewaan')
plt.grid()
st.pyplot(plt)

**Insight:**
- Grafik membantu menunjukkan pola perilaku pengguna secara visual:
    - Rata-rata penyewaan per jam atau bulan memberikan wawasan waktu optimal untuk penyewaan.
    - Rata-rata penyewaan berdasarkan kondisi cuaca memberikan wawasan lingkungan optimal untuk penyewaan.

## Conclusion
- **Apa waktu terbaik untuk memaksimalkan penggunaan sepeda?:**
  - Dari analisis data per jam, ditemukan bahwa waktu pagi hari (07:00-09:00) dan sore hari (17:00-19:00) adalah puncak penyewaan sepeda. Hal ini kemungkinan besar terkait dengan aktivitas perjalanan menuju tempat kerja atau sekolah pada pagi hari dan perjalanan pulang pada sore hari.
  - Dari analisis data harian, musim panas (bulan Juni, Juli, dan Agustus) mencatatkan rata-rata penyewaan tertinggi, yang menunjukkan pengaruh cuaca hangat terhadap peningkatan aktivitas pengguna.
- **Bagaimana pengaruh cuaca terhadap jumlah penyewaan sepeda?:**
  - Cuaca yang cerah atau sebagian berawan (Clear/Partly Cloudy) menghasilkan rata-rata jumlah penyewaan tertinggi. Hal ini menunjukkan bahwa kondisi cuaca yang nyaman sangat mendorong penggunaan sepeda.
  - Cuaca buruk seperti hujan deras atau salju (Heavy Rain/Snow) menyebabkan penurunan signifikan dalam jumlah penyewaan sepeda. Ini mencerminkan bahwa pengguna cenderung menghindari aktivitas luar ruangan dalam kondisi cuaca ekstrem.